# Descrição da tarefa

Esta é uma tarefa de geração de texto (NLG - Natural Language Generation) baseada em modelos de n-gramas -- trigramas, especificamente.

O corpus a ser utilizado é o conjunto da obra em prosa de Machado de Assis, contido no arquivo Machado.zip (na AlunoWeb), com um total aproximado de 1 milhão de palavras.

As etapas do pré-processamento já estão prontas. Quando executadas, vão gerar uma lista de tokens já limpos, normalizados e com os delimitadores de sentenças ($<s>$ e $</s>$) incluídos.

A partir daí, você deve:

1. Gerar os trigramas. Use a função `zip()` para isso.

2. Criar uma função para escolha de um trigrama de início de sentença com o qual você vai começar a geração do texto "original".

3. Incluir novos trigramas até que seja encontrado um final "natural" de sentença ($</s>$).

Neste notebook, além do pré-processamento, são incluídos outros fragmentos de código e sugestões pontuais para ajudar na solução da tarefa.

# Importação de módulos e de funções

A maioria são recursos que já conhecemos bem. Algumas funções foram ligeiramente adaptadas por causa dos dados que serão utilizados no exercício.

In [1]:
import itertools

import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize

# Tokenização de sentenças
def tokenizar_sentencas(txt: str) -> list:
    txt = txt.replace('\n', ' ')
    return sent_tokenize(txt, language='portuguese')


# Tokenização de palavras e símbolos
def tokenizar(txt: str) -> list:
    return word_tokenize(txt, language='portuguese')


def limpar(lista: list) -> list:
    return [i.lower() for i in lista if i.isalpha()]


def achatar(lista: list) -> list:
    return list(itertools.chain(*lista))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
# def ler(nome_arq):
#     arq = open(nome_arq, 'r', encoding='latin-1')  # Essa foi a codificação usada nos textos
#     corpus = arq.read()
#     corpus = corpus.replace('\n', ' ')
#     arq.close()

#     return corpus

# A função de leitura de arquivos que conhecemos foi modificada
# para usar a codificação "latin-1" usada nos textos do corpus Machado
def ler(nome_arquivo) -> str:
    with open(nome_arquivo, encoding='latin-1') as arquivo:
        return arquivo.read()


def delimitar_sents(lst_sents: list) -> list:
    return [['<s>'] + i + ['</s>'] for i in lst_sents]



# Este pipeline foi criado para facilitar a aplicação e ordenação das funções
def pipeline(txt: str) -> list:
    texto = ler(txt)
    sentencas = tokenizar_sentencas(texto)
    sentencas_de_tokens = [limpar(tokenizar(i)) for i in sentencas]
    sentencas_delimitadas_de_tokens = delimitar_sents(sentencas_de_tokens)
    listona_de_tokens = achatar(sentencas_delimitadas_de_tokens)

    return listona_de_tokens

# Carregamento e processamento dos textos

Antes de executar as células a seguir, você deve subir para o drive do Colab o arquivo 'Machado.zip', que está no Moodle.

In [14]:
# Nova biblioteca: permite buscar facilmente nomes de arquivos no drive (HD ou virtual)
import glob

# Descompacta (no drive do Colab) o arquivo zipado
!unzip 'Machado.zip' -d 'machado'

# Recebe uma lista dos nomes de arquivos que foram descompactados na pasta 'machado'
arqs = glob.glob('machado/*.txt')

# Cria uma (grande) lista dos tokens de todas as obras
lst_tokens = list()
for arquivo in arqs:
    print(f'Processando {arquivo}...')
    tokens_da_obra = pipeline(arquivo)
    lst_tokens.append(tokens_da_obra)

# Até aqui, lst_tokens é uma lista de sublistas (cada lista é uma obra).
# Será preciso transformá-la numa lista única, unidimensional:
lst_tokens = achatar(lst_tokens)

Archive:  Machado.zip
  inflating: machado/A Mao e a Luva.txt  
  inflating: machado/Casa Velha.txt  
  inflating: machado/Contos Fluminenses.txt  
  inflating: machado/Dom Casmurro.txt  
  inflating: machado/Esau e Jaco.txt  
  inflating: machado/Helena.txt      
  inflating: machado/Historias Sem Data.txt  
  inflating: machado/Historias da Meia-Noite.txt  
  inflating: machado/Iaia Garcia.txt  
  inflating: machado/Memorial de Aires.txt  
  inflating: machado/Memorias Postumas de Bras Cubas.txt  
  inflating: machado/Paginas Recolhidas.txt  
  inflating: machado/Papeis Avulsos.txt  
  inflating: machado/Quincas Borba.txt  
  inflating: machado/Reliquias de Casa Velha.txt  
  inflating: machado/Ressurreicao.txt  
  inflating: machado/Varias Historias.txt  
Processando machado/Memorial de Aires.txt...
Processando machado/Quincas Borba.txt...
Processando machado/A Mao e a Luva.txt...
Processando machado/Historias Sem Data.txt...
Processando machado/Papeis Avulsos.txt...
Processando mac

# 1. Geração dos trigramas

Você não precisará de nada além dos trigramas nesta tarefa. É só distribuir os tokens em uma grande lista de tuplas representando os trigramas. Use a função `zip()`.

In [22]:
trigrams = list(zip(lst_tokens, lst_tokens[1:], lst_tokens[2:]))

trigrams = list(set(trigrams))

# 2.a. Filtragem dos trigramas

Criados os trigramas, vamos:

1. Filtrar os repetidos (sim, tem muitos!)

2. Dividir os restantes em dois grupos:

    a. Os que são iniciados por < s >. Chamemos a esse grupo de "iniciais".

    b. Os demais serão chamados de "sequentes".

3. Filtrar os sequentes para eliminar os que atrapalhariam a geração de boas sequências de texto: aqueles que têm o delimitador de fim de sentença (< /s >) em qualquer posição que não a última.

In [23]:
iniciais = [tri for tri in trigrams if tri[0] == '<s>']

sequentes = [tri for tri in trigrams if tri[0] != '<s>' and (tri[0] != '</s>' and tri[1] != '</s>')]

# 2.b. Criação de uma função de escolha aleatória

Na geração de texto, é preciso mimetizar a criatividade com alguma aleatoriedade.

Nosso gerador de texto precisará de duas funções de escolhas aleatórias:

1. O trigrama com o qual será iniciada a sequência textual, a ser escolhido aleatoriamente da lista `iniciais`. Vamos atribuir o resultado dessa escolha a uma variável chamada `semente`.

2. Os trigramas de continuidade da sequência textual, escolhidos a partir da lista `sequentes`. Atenção: antes de escolher o trigrama, será preciso gerar uma sublista a partir dos `sequentes` aplicando um filtro que selecione somente as tuplas cuja primeira palavra coincida com a última palavra da sequência já formada. Assim, por exemplo, se a sequência já formada for (< s >, 'bom', 'dia'), você deve filtrar a lista `sequentes` mantendo só as tuplas que comecem por 'dia'. Crie uma função chamada `sequencia` que receba a palavra final do texto já gerado e retorne uma nova tupla cuja primeira palavra coincida com a última do texto. Atenção de novo: essa tupla será formada pelo trigrama, mas não é interessante retornar a primeira palavra do trigrama, já que ela seria uma repetição da última palavra da tupla anterior. Retorne, somente, a segunda e a terceira palavras da tupla.

Um modo prático de gerar escolhas aleatórias em Python é usar o módulo `random`. Ele traz um método `choice` que permite escolher aleatoriamente um item de uma lista qualquer. Segue uma ilustração de uso do método.

In [24]:
import random

def sequencia(palavra_final, sequentes):
    opcoes = [tri for tri in sequentes if tri[0] == palavra_final]
    if not opcoes:
        return None
    escolhido = random.choice(opcoes)
    return escolhido[1], escolhido[2]

# 3. Repetir a escolha de sequentes até chegar a um final de sentença

A ideia é continuar a produzir texto "original" até que seja escolhida uma tupla com uma marcação de fim de sentença (< /s >).

Como não se sabe quando o fim da sentença será encontrado, o modo mais simples de fazer isso é usando um loop com `while`.

Só para deixar claro:

1. Antes de entrar no loop, a `semente` deve iniciar o texto;

2. Dentro do loop `while`, deve estar a chamada à função `sequencia`, que será repetida até que a condição de encontrar o final da sentença seja satisfeita;

3. Sugestão: ao longo do processo, as tuplas selecionadas podem ser adicionadas a uma lista, que será exibida ao final.

In [25]:
semente = random.choice(iniciais)
texto_gerado = list(semente)

while texto_gerado[-1] != '</s>':
    proximo = sequencia(texto_gerado[-1], sequentes)

    if proximo is None:
        break

    texto_gerado.extend(proximo)


# 4. Mostre o resultado para o mundo

Chegou finalmente o momento de exibir a sequência de texto original gerada. Depois de tanto trabalho, vale a pena mostrar alguma coisa mais bonita do que uma lista de palavras, certo? Dê um jeito de fazer com que pareça um texto "natural", com letra maiúscula inicial, escrita cursiva (uma palavra depois da outra, com espaços em branco) e terminando com ponto final.

In [26]:
palavras = [token for token in texto_gerado if token not in ['<s>', '</s>']]

frase = " ".join(palavras)
frase = frase.capitalize() + '.'

print(frase)


Mamãe veja se não atribuía realmente o tinha a escritura qual destes e irão logo a criança teria já saído para um edifício leve um sorriso acanhado e lento quarenta cabeças e a responsabilidade da perda do meu tio sabe as coisas que serviam de resplendor maior da família poucas e silenciosas e não pude ter conhecido a mariposa que a viu o ano justamente em quando interrogativa queria ouvir o artigo político o ser tenta dominar a sensação de perdoar quer catar argumentos que empreguei algumas reflexões que sinto que me não arrependo de o comborço.
